## Обрабатываем титульную страницу.

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    %reset

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    %run common_info.ipynb

### Извлекаем содержимое титульной страницы.

In [ ]:
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
import re

 
def parseTitlePage(graduationWorkPDFFile):
    laparams = LAParams()
    setattr(laparams, 'all_texts', True)
    
    with open(graduationWorkPDFFile, 'rb') as fh:
        titlePage = next(PDFPage.get_pages(fh, caching=True, check_extractable=True))
            
        resourceManager = PDFResourceManager()
        fakeFileHandle = io.StringIO()
        # codec = 'windows-1252'
        converter = TextConverter(resourceManager, fakeFileHandle, laparams=laparams)
        pageInterpreter = PDFPageInterpreter(resourceManager, converter)
        pageInterpreter.process_page(titlePage)
 
        text = fakeFileHandle.getvalue()
        while text.find('  ') != -1:
            text = text.replace('  ', ' ')
        while text.find(' \n') != -1:
            text = text.replace(' \n', '\n')
        while text.find('\n\n') != -1:
            text = text.replace('\n\n', '\n')
 
        converter.close()
        fakeFileHandle.close()
        
        return text

In [ ]:
titlePageText = parseTitlePage(graduationWorkPDFFile)

if __name__ == '__main__' and '__file__' not in globals():
    print(titlePageText)

### Обрабатываем содержимое титульной страницы.

In [ ]:
def processTitlePage(titlePageText):
    titlePageTemplate  = "Московский государственный университет имени М.В. Ломоносова\n"
    titlePageTemplate += "Факультет вычислительной математики и кибернетики\n"
                         # Название кафедры
                         # Фамилия Имя Отчество студента
                         # Название работы, 1-2-3 строки
    titlePageTemplate += "ВЫПУСКНАЯ КВАЛИФИКАЦИОННАЯ РАБОТА\n"
                         # или "МАГИСТЕРСКАЯ ДИССЕРТАЦИЯ"
    titlePageTemplate += "Научный руководитель:\n"
                         # должность научного руководителя
                         # Фамилия Имя Отчество научного руководителя
    titlePageTemplate += "Москва," # + *год защиты работы*

    titlePageDict = {"university": "Московский государственный университет имени М.В. Ломоносова",
                     "faculty": "Факультет вычислительной математики и кибернетики",
                     # Название кафедры
                     # Фамилия Имя Отчество студента
                     # Название работы, 1-2-3 строки
                     "workType": "ВЫПУСКНАЯ КВАЛИФИКАЦИОННАЯ РАБОТА",
                     # "workType": "МАГИСТЕРСКАЯ ДИССЕРТАЦИЯ\n",
                     "scientificDirectorMarker": "Научный руководитель:",
                     # должность научного руководителя
                     # Фамилия Имя Отчество научного руководителя
                     "cityAndYear": "Москва,"} # + *год защиты работы*

    for key in titlePageDict.keys():
        if key == "cityAndYear":
            titlePageText = titlePageText[0:titlePageText.find(titlePageDict[key])-1]

    titlePageTextByLine = titlePageText.split('\n')
    titlePageDict['department'] = titlePageTextByLine[2]
    titlePageDict['nameAndSurname'] = titlePageTextByLine[3]
    titlePageDict['scitificDirectorNameAndSurname'] = titlePageTextByLine[-1]
    titlePageDict['scitificDirectorPosition'] = titlePageTextByLine[-2]
    titlePageTextByLine = titlePageTextByLine[4:-4]

    for workTitleLine in titlePageTextByLine:
        if 'workTitle' not in titlePageDict.keys():
            titlePageDict['workTitle'] = workTitleLine
        else:
            titlePageDict['workTitle'] += ' ' + workTitleLine

    return titlePageDict

In [ ]:
from pprint import pprint

titlePageDict = processTitlePage(titlePageText)

if __name__ == '__main__' and '__file__' not in globals():
    pprint(titlePageDict)